# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 11, 11, 12, 10

while True:
    dt = datetime.now()
    if dt.month >= month:
        if dt.day > day or dt.month > month:
            break
        if dt.day == day and (dt.hour > hour or (dt.hour == hour and dt.minute > minute)):
            break
    sleep(60)

# Parameter optimization

In [1]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['PriceChange', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 100
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'PriceChange': {"low_price_quantile": [1 * (i +  1) for i in range(20)]}, 
                'LinearReg': {'timeperiod': [6, 8, 10], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 15m/1h, trade type is sell
Number of combinations is 60


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [02:10<00:00,  2.18s/it]


# Check local statistics

In [2]:
# 15m/4h
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('pct_right_forecast_avg', ascending=False).head(20)

,pattern,PriceChange_low_price_quantile,LinearReg_timeperiod,LinearReg_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank
25,PriceChange_LinearReg,9,8,0,91.67,100.00,83.33,91.67,91.67,91.67,83.33,83.33,75.00,83.33,83.33,83.33,58.33,75.00,66.67,66.67,75.00,66.67,58.33,58.33,58.33,66.67,66.67,75.00,-2.07,-3.03,-3.22,-3.38,-3.92,-2.75,-2.10,-2.49,-3.38,-2.78,-4.07,-4.83,-2.00,-2.63,-1.79,-1.83,-1.71,-1.74,-1.26,-2.17,-2.14,-2.18,-2.26,-3.30,12,76.388750,-2.626250,76.665000,-31.515000
19,PriceChange_LinearReg,7,8,0,90.00,100.00,80.00,90.00,90.00,90.00,80.00,80.00,70.00,80.00,80.00,80.00,60.00,70.00,60.00,60.00,70.00,60.00,60.00,60.00,60.00,70.00,70.00,80.00,-2.25,-3.03,-2.53,-3.38,-3.92,-2.75,-2.10,-2.49,-3.38,-2.78,-4.07,-4.83,-2.00,-2.06,-1.56,-1.83,-1.71,-1.74,-1.26,-2.17,-2.14,-2.18,-2.26,-3.30,10,74.583333,-2.571667,45.833333,-25.716667
22,PriceChange_LinearReg,8,8,0,90.91,100.00,81.82,90.91,90.91,90.91,81.82,81.82,72.73,81.82,81.82,81.82,54.55,72.73,63.64,63.64,72.73,63.64,54.55,54.55,54.55,63.64,63.64,72.73,-2.22,-2.84,-2.95,-2.57,-3.29,-2.19,-2.06,-1.84,-2.39,-2.02,-2.55,-2.48,-1.46,-2.25,-1.42,-1.41,-1.52,-1.30,-0.69,-1.30,-1.41,-2.02,-1.63,-2.83,11,74.245000,-2.026667,46.695000,-22.293333
28,PriceChange_LinearReg,10,8,0,92.86,92.86,78.57,92.86,92.86,85.71,78.57,78.57,71.43,78.57,78.57,78.57,57.14,71.43,71.43,64.29,71.43,64.29,57.14,57.14,57.14,64.29,64.29,71.43,-1.67,-2.49,-2.25,-2.33,-3.13,-2.75,-2.10,-2.42,-2.58,-2.45,-2.68,-2.49,-1.75,-1.87,-1.28,-1.74,-1.71,-1.74,-1.26,-1.91,-2.14,-2.18,-2.26,-3.06,14,73.810000,-2.176667,53.340000,-30.473333
16,PriceChange_LinearReg,6,8,0,85.71,100.00,71.43,85.71,85.71,85.71,71.43,71.43,57.14,71.43,71.43,71.43,42.86,57.14,57.14,57.14,71.43,71.43,71.43,71.43,71.43,85.71,85.71,85.71,-2.22,-2.84,-1.56,-4.19,-4.54,-2.19,-2.06,-0.94,-0.87,-0.98,-1.83,-0.70,0.40,-0.76,-0.98,-1.41,-1.52,-2.21,-3.45,-3.03,-3.36,-6.84,-6.43,-4.57,7,73.213333,-2.461667,22.493333,-17.231667
31,PriceChange_LinearReg,11,8,0,93.33,93.33,80.00,93.33,93.33,86.67,80.00,80.00,73.33,80.00,80.00,80.00,60.00,73.33,73.33,60.00,66.67,60.00,53.33,53.33,53.33,60.00,60.00,66.67,-1.41,-2.13,-1.56,-2.09,-2.98,-2.19,-2.06,-1.84,-2.39,-2.02,-2.55,-2.48,-1.46,-1.48,-1.14,-1.41,-1.52,-1.30,-0.69,-1.30,-1.41,-2.02,-1.63,-2.83,15,73.054583,-1.828750,45.818750,-27.431250
30,PriceChange_LinearReg,11,6,0,94.12,88.24,76.47,88.24,82.35,76.47,82.35,76.47,70.59,82.35,76.47,76.47,58.82,70.59,70.59,58.82,64.71,58.82,52.94,52.94,47.06,52.94,52.94,58.82,-1.37,-1.89,-1.16,-1.80,-2.49,-1.59,-2.13,-1.84,-2.78,-2.88,-2.27,-2.16,-1.46,-1.48,-1.14,-1.41,-1.50,-0.86,-0.08,-0.70,0.49,-1.30,-1.30,-2.32,17,69.607500,-1.559167,-6.672500,-26.505833
24,PriceChange_LinearReg,9,6,0,92.31,92.31,76.92,84.62,76.92,76.92,84.62,76.92,69.23,84.62,76.92,76.92,53.85,69.23,61.54,61.54,69.23,61.54,53.85,53.85,46.15,53.85,53.85,61.54,-1.41,-2.13,-1.56,-2.09,-2.98,-1.59,-2.13,-1.84,-3.55,-3.53,-2.27,-2.16,-1.46,-2.25,-1.42,-1.41,-1.50,-0.86,-0.08,-0.70,0.49,-1.30,-1.30,-2.32,13,69.552083,-1.722917,-5.822917,

# Save new config data to config file

In [3]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['PriceChange', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'PriceChange': {"low_price_quantile": [10]}, 
              'LinearReg': {'timeperiod': [8], 'low_bound': [0]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [3]:
from glob import glob

ttype = 'sell'
pattern = ['PriceChange']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['PriceChange_low_price_quantile']).agg({'pct_right_forecast_avg': 'mean',
                                                                   'pct_price_diff_avg': 'mean',
                                                                   'forecast_rank': 'mean', 
                                                                   'price_rank': 'mean', 
                                                                   'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                                        ascending=False)
total_stat.head(20)

,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank,forecasts_num
PriceChange_low_price_quantile,,,,,
20,72.930243,-1.306944,-879.743681,-157.711944,3071
19,73.131424,-1.325521,-881.025174,-153.987604,3038
17,73.016458,-1.380833,-906.773576,-147.793403,2972
18,72.718576,-1.329062,-908.998681,-150.019097,3005
16,73.201181,-1.421007,-909.151806,-143.919722,2933
15,73.250174,-1.485451,-924.197604,-142.200174,2911
12,74.196285,-1.676250,-927.744236,-130.194201,2805
13,73.203299,-1.593021,-954.058576,-131.837604,2836
11,73.823993,-1.782292,-954.826875,-126.921111,2770
